```{contents}
```
## AgentExecutor 


`AgentExecutor` is the **runtime engine** that **drives an agent’s execution loop** in LangChain.

> An agent defines *what to do* (reasoning + tool selection).
> `AgentExecutor` defines *how it runs* (iterations, tool execution, stopping, safety).

Without `AgentExecutor`, an agent **cannot actually run**.

---

### Responsibilities of AgentExecutor

`AgentExecutor` is responsible for:

1. **Invoking the agent**
2. **Executing tool calls**
3. **Feeding tool results back to the agent**
4. **Managing the agent loop**
5. **Stopping execution safely**
6. **Returning the final answer**

---

### High-Level Execution Flow

```
User Input
   ↓
AgentExecutor
   ↓
Agent (decides next step)
   ↓
Tool Call (if any)
   ↓
Tool Execution
   ↓
Observation
   ↓
Agent (next step or final answer)
   ↓
AgentExecutor returns output
```

This loop repeats until a stop condition is met.

---

### Agent vs AgentExecutor (Critical Distinction)

| Component     | Role                       |
| ------------- | -------------------------- |
| Agent         | Reasoning & decision logic |
| AgentExecutor | Orchestration & execution  |

**Agent = brain**
**AgentExecutor = engine**

---

### Minimal Example

#### Create an Agent



In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.tools import tool
from langchain_classic.agents import create_openai_tools_agent

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


@tool
def ticket_count(source: str) -> int:
    """Return number of tickets from a source"""
    return 128

agent = create_openai_tools_agent(
    llm=chat,
    tools=[ticket_count],
    prompt=prompt
)




At this stage:

* The agent **cannot run by itself**

---

#### Wrap with AgentExecutor



In [3]:
from langchain_classic.agents import AgentExecutor

executor = AgentExecutor(
    agent=agent,
    tools=[ticket_count],
    verbose=True
)


Now the agent is **executable**.

---

#### Run the Agent


In [4]:
result = executor.invoke(
    {"input": "How many tickets are there in Jira?"}
)

print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `ticket_count` with `{'source': 'Jira'}`


128There are 128 tickets in Jira.

> Finished chain.
There are 128 tickets in Jira.



---

### What Happens Internally (Step-by-Step)

1. `AgentExecutor` receives user input
2. Calls the agent to decide the next action
3. If the agent requests a tool:

   * Executor validates the tool
   * Executes the tool
4. Tool output becomes an **observation**
5. Observation is added to `agent_scratchpad`
6. Agent is called again
7. Loop continues until:

   * Agent returns `Final Answer`
   * Or max steps reached

---

### Key Parameters of AgentExecutor

#### `agent`

The reasoning logic (ReAct, OpenAI tools agent, etc.)

---

#### `tools`

List of allowed tools the executor can run.

```python
tools=[ticket_count, sla_threshold]
```

Only these tools can be executed.

---

#### `verbose`

```python
verbose=True
```

Prints:

* Agent thoughts (if allowed)
* Tool calls
* Observations

Used for **debugging only**.

---

#### `max_iterations`

```python
max_iterations=5
```

Prevents infinite loops.

**Production best practice:** always set this.

---

#### `early_stopping_method`

Controls behavior when max iterations are hit.

Options:

* `"force"` → stop immediately
* `"generate"` → generate best possible answer

---

#### `return_intermediate_steps`

```python
return_intermediate_steps=True
```

Returns tool calls and observations along with final output.

Useful for:

* Debugging
* Auditing
* Observability

---

#### Output Structure

Default output:

```python
{
  "output": "There are 128 tickets in Jira."
}
```

With `return_intermediate_steps=True`:

```python
{
  "output": "...",
  "intermediate_steps": [
    (tool_call, observation),
    ...
  ]
}
```

---

#### AgentExecutor and `agent_scratchpad`

* `AgentExecutor` manages `intermediate_steps`
* Converts them into `agent_scratchpad`
* Injects them into the prompt automatically

You **never** manage this manually.

---

#### AgentExecutor vs Chains

| Aspect       | Chain | AgentExecutor |
| ------------ | ----- | ------------- |
| Control flow | Fixed | Dynamic       |
| Tool usage   | ❌     | ✅             |
| Looping      | ❌     | ✅             |
| Reasoning    | ❌     | ✅             |

---

### AgentExecutor vs LangGraph

| Aspect            | AgentExecutor | LangGraph |
| ----------------- | ------------- | --------- |
| Control           | LLM-driven    | Explicit  |
| Predictability    | Medium        | High      |
| Debugging         | Harder        | Easier    |
| Production safety | Medium        | High      |

**Rule:**

* Use `AgentExecutor` for flexibility
* Use LangGraph for mission-critical workflows

---

### Common Mistakes

### Forgetting AgentExecutor

❌ Agent never runs

### Not setting `max_iterations`

❌ Infinite loops

### Exposing verbose logs to users

❌ Security risk

### Too many tools

❌ Tool confusion

---

### Production Best Practices

* Always set `max_iterations`
* Keep tool list minimal
* Separate read vs write tools
* Log intermediate steps securely
* Use low temperature
* Add human approval for destructive actions

---

### Interview-Ready Summary

> “`AgentExecutor` is the orchestration layer in LangChain that runs an agent’s reasoning loop, executes tool calls, manages intermediate steps, enforces stopping conditions, and returns the final output.”

---

### Rule of Thumb

* **Agent defines decisions**
* **AgentExecutor executes them**
* **No executor → no agent**
* **Production safety → strict limits**

If you want next:

* Debugging agents with `return_intermediate_steps`
* Migrating AgentExecutor logic to LangGraph
* AgentExecutor vs LCEL chains
